# Analyze DTI

Useful links and usage examples:
Convert ANTs H5 warp files to fsl format
I used the c(3): 
https://www.jiscmail.ac.uk/cgi-bin/webadmin?A2=fsl;c21935f9.1901

/Users/ranigera/Downloads/Convert3DGUI.app/Contents/bin/c3d_affine_tool -ref sub-175_desc-preproc_T1w.nii.gz -src sub-175_first_processed_B0_before.nii.gz sub-175_first_processed_B0_reg_to_native_T1_before.mat -fsl2ras -oitk ANTS_ref_T1.txt

antsApplyTransforms -i sub-175_first_processed_B0_before.nii.gz -r ${FSLDIR}/data/standard/MNI152_T1_2mm.nii.gz -o B0_in_mni_fromT1regFINAL.nii.gz -t sub-175_from-T1w_to-MNI152NLin2009cAsym_mode-image_xfm.h5 -t ANTS_ref_T1.txt 

In [13]:
# import packages
# -----------------------------------------------------------------------------
import pandas as pd
import numpy as np
import os
import glob
import subprocess
import sys
import json
import platform
import socket

In [14]:
# parameters
# -----------------------------------------------------------------------------
MANUALLY_EXCLUDE_SUBJECTS = []

running_on = 'server' if 'Linux' in platform.system() else 'my_mac'

if running_on == 'my_mac':
    data_path = '/Users/ranigera/Dropbox/DTI_tests'
    preproc_path = '/Users/ranigera/Dropbox/DTI_tests/preproc'
    dti_path = '/Users/ranigera/Dropbox/DTI_tests/dti'
    stats_path = '/Users/ranigera/Dropbox/DTI_tests/stats_alt_reg'
    models_path = stats_path + '/models'
    launch_files_path = '/Users/ranigera/Dropbox/DTI_analysis/launch_files'
else:
    data_path = '/export2/DATA/HIS/HIS_server/BIDS'
    preproc_path = '/export2/DATA/HIS/HIS_server/analysis/dwi_data/preproc'
    dti_path = '/export2/DATA/HIS/HIS_server/analysis/dwi_data/dti'
    stats_path = '/export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg'
    models_path = stats_path + '/models'
    launch_files_path = '/export2/DATA/HIS/HIS_server/codes_dwi/launch_files'
    behav_data_path = '/export2/DATA/HIS/HIS_server/analysis/behavior_analysis_output/my_databases/txt_data'

fmriPrepAnatomyDerivatives_path = data_path + '/derivatives/fmriprep'

expectedVolums = {
    'AP': 69,
    'PA' : 7,
    }
expectedB0s_indxs = {
    'AP_before': [0, 1, 18, 35, 52],
    'PA_before': [0, 2, 3, 4, 5, 6],
    'AP_after': [0, 1, 18, 35, 52],
    'PA_after': [0, 2, 3, 4, 5, 6]
    }

n_cores_TOPUP = 2

# setting EDDY stuff:
EDDY_command = 'eddy_openmp' if running_on == 'server' else 'eddy' # for the boost server
EDDY_command = 'eddy_cuda10.2' if running_on == 'server' else 'eddy' # for the cheshire server
ssh_command_for_cheshire_server = 'ssh shirangera@cheshire.tau.ac.il' if 'boost' in socket.gethostname() else ''

n_expected_EDDY_output_files = 13
n_cores_EDDY = 4 # relevant only for running using files (currently disabled as I run it on cheshire's GPU)

n_expected_DTIFIT_output_files = 10

masks_paths = {
    'Caudate': '/export2/DATA/HIS/HIS_server/DTI_assitance_files/masks/Harvard-Oxford/CaudateHead_Y-larger-than-1-mask.nii.gz',
    'Putamen': '/export2/DATA/HIS/HIS_server/DTI_assitance_files/masks/Harvard-Oxford/Putamen-mask.nii.gz',
    'vmPFC':   '/export2/DATA/HIS/HIS_server/DTI_assitance_files/masks/Harvard-Oxford/vmPFC-mask.nii.gz',
}

In [3]:
# Define functions
# -----------------------------------------------------------------------------
def createSubjectScansBaseNames(subjFolder, data_path):
    sub = int(subjFolder.split("-",1)[1])
    group = '1day' if sub < 200 else '3day'
    last_sess = group[0]
    DWI_path_before = os.path.join(data_path, subjFolder, 'ses-1/dwi/')
    DWI_path_after = os.path.join(data_path, subjFolder, f'ses-{last_sess}/dwi/')
    scansBaseNames = {
        'AP_before': f'{os.path.join(DWI_path_before, "sub-" + str(sub) + "_ses-1_acq-ap_run-01_dwi")}',
        'PA_before' : f'{os.path.join(DWI_path_before, "sub-" + str(sub) + "_ses-1_acq-pa_run-01_dwi")}',
        'AP_after' : f'{os.path.join(DWI_path_after, "sub-" + str(sub) + "_ses-" + last_sess + "_acq-ap_run-02_dwi")}',
        'PA_after' : f'{os.path.join(DWI_path_after, "sub-" + str(sub) + "_ses-" + last_sess + "_acq-pa_run-02_dwi")}'
        }
    return scansBaseNames

def get_sub_B0_files(subjFolder, scansBaseNames, B0s_indxs):
    sub_B0s_files = []
    for scan in scansBaseNames.keys():
        for B0ind in B0s_indxs[scan]:
            B0_file_name = os.path.join(preproc_path, subjFolder, subjFolder + '_' + scan + "_b0_volInd-" + str(B0ind) + ".nii.gz")
            sub_B0s_files.append(B0_file_name)
    return sub_B0s_files


In [4]:
# Get folders and remove excluded subjects
# -----------------------------------------------------------------------------
print('>> Get sub folders')
subjFolders = [el for el in os.listdir(data_path) if 'sub' in el]

if running_on == 'my_mac':
    print('>> Get exclusion list')
    with open('/Users/ranigera/Google_Drive_TAU/Experiments/HIS_STUDY/Analysis/codes/paths_and_vars.py') as txtFile:
        txt = txtFile.read()
    participantsToExclude = [int(el) for el in txt.split('participantsToExclude = [')[1].split(']')[0].replace('\n','').replace('\n','').replace("'","").split(',')]

    print('>> Remove sub folders of excluded participants in case they are there')
    subjFolders = [el for el in subjFolders if int(el.split('-')[1]) not in participantsToExclude]

if MANUALLY_EXCLUDE_SUBJECTS:
    subjFolders = [el for el in subjFolders if int(el.split('-')[1]) not in MANUALLY_EXCLUDE_SUBJECTS]
    
subjFolders.sort()

>> Get sub folders


## Check for missing scans or wrong phase encoding directions for ALL SUBJECTS 

In [5]:
print ('>> Verify that all the scans exist and that the phase encoding directions are as they should.')
subjectsWithAProblem = []
for subjFolder in subjFolders:
    sub = int(subjFolder.split("-",1)[1])
    scansBaseNames = createSubjectScansBaseNames(subjFolder, data_path)
    for scan in scansBaseNames.keys():
        # print(scansBaseNames[scan] + '.json')
        # print(scanData['PhaseEncodingDirection'])
        if not os.path.exists(scansBaseNames[scan] + '.json'):
            subjectsWithAProblem.append(sub)
            print(' *** Scan not found: ' + scansBaseNames[scan] + '.json')
            continue
        with open(scansBaseNames[scan] + '.json') as json_file:        
            scanData = json.load(json_file)
            if ('acq-ap_' in scansBaseNames[scan] and scanData['PhaseEncodingDirection'] != 'j-') or \
                ('acq-pa_' in scansBaseNames[scan] and scanData['PhaseEncodingDirection'] != 'j'):
                subjectsWithAProblem.append(sub)
                print(' *** There is a problem with the scanning directions: ' + scansBaseNames[scan] + '.json is defined as ' + scanData['PhaseEncodingDirection'] + '.')
                continue

subjectsWithAProblem = list(set(subjectsWithAProblem))
subjectsWithAProblem.sort()

>> Verify that all the scans exist and that the phase encoding directions are as they should.
 *** There is a problem with the scanning directions: /export2/DATA/HIS/HIS_server/BIDS/sub-204/ses-3/dwi/sub-204_ses-3_acq-pa_run-02_dwi.json is defined as j-.
 *** There is a problem with the scanning directions: /export2/DATA/HIS/HIS_server/BIDS/sub-207/ses-1/dwi/sub-207_ses-1_acq-pa_run-01_dwi.json is defined as j-.
 *** There is a problem with the scanning directions: /export2/DATA/HIS/HIS_server/BIDS/sub-209/ses-1/dwi/sub-209_ses-1_acq-pa_run-01_dwi.json is defined as j-.
 *** Scan not found: /export2/DATA/HIS/HIS_server/BIDS/sub-255/ses-1/dwi/sub-255_ses-1_acq-pa_run-01_dwi.json
 *** Scan not found: /export2/DATA/HIS/HIS_server/BIDS/sub-259/ses-3/dwi/sub-259_ses-3_acq-ap_run-02_dwi.json
 *** Scan not found: /export2/DATA/HIS/HIS_server/BIDS/sub-259/ses-3/dwi/sub-259_ses-3_acq-pa_run-02_dwi.json


In [6]:
# Remove subjects with a problem
# -----------------------------------------------------------------------------
print('>> Remove subjects with a problem and sort the list')
subjFolders =[el for el in subjFolders if int(el.split('-')[1]) not in subjectsWithAProblem]
subjFolders.sort()

>> Remove subjects with a problem and sort the list


## Get b0 volume indices and perform bval & bvec QA

In [7]:

for subjFolder in subjFolders:
    scansBaseNames = createSubjectScansBaseNames(subjFolder, data_path)
    print('> Verify that data points in the bval files is as expected.')
    if pd.read_csv(scansBaseNames['AP_before'] + '.bval', header=None, sep=' ').T.shape[0] != expectedVolums['AP'] or \
        pd.read_csv(scansBaseNames['PA_before'] + '.bval', header=None, sep=' ').T.shape[0] != expectedVolums['PA'] or \
        pd.read_csv(scansBaseNames['AP_after'] + '.bval', header=None, sep=' ').T.shape[0] != expectedVolums['AP'] or \
        pd.read_csv(scansBaseNames['PA_after'] + '.bval', header=None, sep=' ').T.shape[0] != expectedVolums['PA']:
            print(f' *** The number of data points in the bval for one of the scans for subjetc {sub} is not as expected.')
            raise Exception(f'The number of data points in the bval for one of the scans for subjetc {sub} is not as expected.')

    print('> Verify that data points in the bvec files is as expected.')
    if pd.read_csv(scansBaseNames['AP_before'] + '.bvec', header=None, sep=' ').T.shape[0] != expectedVolums['AP'] or \
        pd.read_csv(scansBaseNames['PA_before'] + '.bvec', header=None, sep=' ').T.shape[0] != expectedVolums['PA'] or \
        pd.read_csv(scansBaseNames['AP_after'] + '.bvec', header=None, sep=' ').T.shape[0] != expectedVolums['AP'] or \
        pd.read_csv(scansBaseNames['PA_after'] + '.bvec', header=None, sep=' ').T.shape[0] != expectedVolums['PA']:
            print(f' *** The number of data points in the bvec for one of the scans for subjetc {sub} is not as expected.')
            raise ValueError(f'The number of data points in the bvec for one of the scans for subjetc ' + str(sub) + ' is not as expected.')

    print('> Extract B0s:')
    B0s_indxs = {}
    for scan in scansBaseNames.keys():
        B0s=pd.read_csv(scansBaseNames[scan] + '.bval', header=None, sep=' ').T
        B0s.columns = ['bval']
        B0s_indxs[scan] = list(B0s[B0s.bval < 20].index)

    print('> Verify that b0 quantity and indices are as expected.')
    if B0s_indxs != expectedB0s_indxs:
        print(f' *** The indices of the b0s for one of the scans for subjetc {sub} are not as expected.')
        raise ValueError(f'The indices of the b0s for one of the scans for subjetc ' + str(sub) + ' are not as expected.')


> Verify that data points in the bval files is as expected.
> Verify that data points in the bvec files is as expected.
> Extract B0s:
> Verify that b0 quantity and indices are as expected.
> Verify that data points in the bval files is as expected.
> Verify that data points in the bvec files is as expected.
> Extract B0s:
> Verify that b0 quantity and indices are as expected.
> Verify that data points in the bval files is as expected.
> Verify that data points in the bvec files is as expected.
> Extract B0s:
> Verify that b0 quantity and indices are as expected.
> Verify that data points in the bval files is as expected.
> Verify that data points in the bvec files is as expected.
> Extract B0s:
> Verify that b0 quantity and indices are as expected.
> Verify that data points in the bval files is as expected.
> Verify that data points in the bvec files is as expected.
> Extract B0s:
> Verify that b0 quantity and indices are as expected.
> Verify that data points in the bval files is as 

## Preprocessing

#### Create pre-processing folder

In [8]:
print('> Create preprocessing folders')
for subjFolder in subjFolders:
    try:
        os.makedirs(os.path.join(preproc_path, subjFolder), exist_ok=False)
        print('>> Created folder: ' + os.path.join(preproc_path, subjFolder))
    except:
        pass

> Create preprocessing folders


### (1) TOPUP
* A tool for estimating and correcting susceptibility induced distortions

#### Extract B0 volumes

In [9]:
print('> Extract B0s (using the fslroi):')
for subjFolder in subjFolders:
    scansBaseNames = createSubjectScansBaseNames(subjFolder, data_path)
    for scan in scansBaseNames.keys():
        for B0ind in B0s_indxs[scan]:
            B0_file_name = os.path.join(preproc_path, subjFolder, subjFolder + '_' + scan + "_b0_volInd-" + str(B0ind) + ".nii.gz")
            if not os.path.isfile(f"{B0_file_name}"):
                print(f'>> runs: fslroi {scansBaseNames[scan]}.nii.gz {B0_file_name} {B0ind} 1')
                os.system(f'fslroi {scansBaseNames[scan]}.nii.gz {B0_file_name} {B0ind} 1')
print(f">> COMPLETED.")

> Extract B0s (using the fslroi):
>> COMPLETED.


#### Merge B0 volumes for TOPUP

In [10]:
print('> Merge B0s for each session (using the fslmerge):')
#print([scan for scan in sub_B0s_files if 'AP_before' in scan])
#print([scan for scan in sub_B0s_files if 'PA_before' in scan])
for subjFolder in subjFolders:
    # get the sub_B0s_files for the current subject:
    scansBaseNames = createSubjectScansBaseNames(subjFolder, data_path)
    sub_B0s_files = get_sub_B0_files(subjFolder, scansBaseNames, B0s_indxs)
    # merge the B0s for the current subject for each time (before/after):
    for time in ['before', 'after']:
        output_base_name = f'{os.path.join(preproc_path, subjFolder,subjFolder)}_AP_PA_{time}_b0s'
        if not os.path.isfile(f"{output_base_name}.nii.gz"):
            print(f'>> runs: fslmerge -t {output_base_name}' + ' ' + ' '.join([scan for scan in sub_B0s_files if f'AP_{time}' in scan]) + ' ' + ' '.join([scan for scan in sub_B0s_files if f'PA_{time}' in scan]))
            os.system(f'fslmerge -t {output_base_name}' + ' ' + ' '.join([scan for scan in sub_B0s_files if f'AP_{time}' in scan]) + ' ' + ' '.join([scan for scan in sub_B0s_files if f'PA_{time}' in scan]))
print(f">> COMPLETED.")

> Merge B0s for each session (using the fslmerge):
>> COMPLETED.


#### Generate the acqparams.txt files

In [11]:
print('> Create the acqparams.txt files (one per session [''before'' and ''after''])')

for subjFolder in subjFolders:
    # get the sub_B0s_files for the current subject:
    scansBaseNames = createSubjectScansBaseNames(subjFolder, data_path)
    sub_B0s_files = get_sub_B0_files(subjFolder, scansBaseNames, B0s_indxs)

    # first get the totalReadoutTime from the json files
    total_readout_time = {}
    for scanBaseName in scansBaseNames.keys():
        with open(scansBaseNames[scanBaseName] + '.json') as json_file:        
            scanData = json.load(json_file)
            total_readout_time[scanBaseName] = scanData['TotalReadoutTime']

    for time in ['before', 'after']:
        acqPars=[f'0 -1 0 {total_readout_time[f"AP_{time}"]}' for scan in sub_B0s_files if f'AP_{time}' in scan] + [f'0 1 0 {total_readout_time[f"PA_{time}"]}' for scan in sub_B0s_files if f'PA_{time}' in scan]
        # write a list of strings to a file (one string per line):
        with open(os.path.join(preproc_path, subjFolder, subjFolder + f'_{time}_acqparams.txt'), 'w') as f:
            for item in acqPars:
                f.write("%s\n" % item)
print(f">> COMPLETED.")

> Create the acqparams.txt files (one per session [before and after])
>> COMPLETED.


#### Run TOPUP

Create TOPUP launch files

In [12]:
print('> Create the TOPUP launch files (one per subject [''before'' and ''after''] together)')
for subjFolder in subjFolders:

    launch_file = os.path.join(launch_files_path, 'TOPUP_' + subjFolder + '_launch.txt')
    
    if not os.path.isfile(launch_file):
        with open(launch_file, 'w') as f:
            for time in ['before', 'after']:
                    print(f">> This command was written to {launch_file}:\ntopup --imain={os.path.join(preproc_path, subjFolder,subjFolder)}_AP_PA_{time}_b0s \\\n\
                            --datain={os.path.join(preproc_path, subjFolder, subjFolder + f'_{time}_acqparams.txt')} \\\n\
                            --config=b02b0.cnf \\\n\
                            --out={os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s \\\n\
                            --iout={os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_iout \\\n\
                            --fout={os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_fout \n\
                        ")
                    f.write(f"topup --imain={os.path.join(preproc_path, subjFolder,subjFolder)}_AP_PA_{time}_b0s --datain={os.path.join(preproc_path, subjFolder, subjFolder + f'_{time}_acqparams.txt')} --config=b02b0.cnf --out={os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s --iout={os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_iout --fout={os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_fout\n")
print(f">> COMPLETED.")

> Create the TOPUP launch files (one per subject [before and after] together)
>> COMPLETED.


Run TOPUP launch files

In [13]:
print('> Run the TOPUP launch files (one per subject [''before'' and ''after''] together)')

for subjFolder in subjFolders:

    launch_file = os.path.join(launch_files_path, 'TOPUP_' + subjFolder + '_launch.txt')

    # check if all expected output files exist:
    expected_TOPUP_out_files_exist = []
    for time in ['before', 'after']:
        expected_TOPUP_out_files_exist += [os.path.isfile(f"{os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_fieldcoef.nii.gz"),
                                        os.path.isfile(f"{os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_fout.nii.gz"),
                                        os.path.isfile(f"{os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_iout.nii.gz"),
                                        os.path.isfile(f"{os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_movpar.txt")]

    # Run the launch file if not all output files present:
    if not all(expected_TOPUP_out_files_exist):
                print(f">> Running: launch -s {launch_file} -j schonberglab -p {n_cores_TOPUP} -r inf")
                os.system(f"launch -s {launch_file} -j schonberglab -p {n_cores_TOPUP} -r inf")         
                
print(f">> COMPLETED.")

> Run the TOPUP launch files (one per subject [before and after] together)
>> COMPLETED.


### (2) EDDY
* A tool for correcting Eddy currents (and motion)

#### Average TOPUP corrected (unwarped) B0 volums (the .iout file)

In [14]:
print(f">> Average TOPUP corrected (unwarped) B0 volums (the .iout file). - runs seperately for each time [''before'' and ''after'']")
for subjFolder in subjFolders:
      for time in ['before', 'after']:
            output_file_name = f"{os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_iout_avg.nii.gz"
            if not os.path.isfile(f"{output_file_name}"):
                  print(f">> runs: fslmaths {os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_iout -Tmean {output_file_name}")
                  os.system(f"fslmaths {os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_iout -Tmean {output_file_name}")
print(f">> COMPLETED.")

>> Average TOPUP corrected (unwarped) B0 volums (the .iout file). - runs seperately for each time [''before'' and ''after'']
>> COMPLETED.


#### Run BET on the averaged B0s volume

In [15]:
print(f">> Extarct the brain form the averaged B0s volume - runs seperately for each time [''before'' and ''after'']")
for subjFolder in subjFolders:
      for time in ['before', 'after']:
            output_base_name = f"{os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_iout_avg_brain"
            if not os.path.isfile(f"{output_base_name}.nii.gz") or not os.path.isfile(f"{output_base_name}_mask.nii.gz"):
                  print(f">> runs: bet {os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_iout_avg {output_base_name} -m -f 0.2")
                  os.system(f"bet {os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_iout_avg {output_base_name} -m -f 0.2")
print(f">> COMPLETED.")

>> Extarct the brain form the averaged B0s volume - runs seperately for each time [''before'' and ''after'']
>> COMPLETED.


#### Create index.txt file
This file maps the volumes in the main DTI data to the relevant line in the acqparams.txt and in the movpar.txt (assessed movement parameters from TOPUP) files
* details in: https://www.youtube.com/watch?v=1T1cRnX7MpA

In [16]:
print(f">> create an index.txt file for each subject for each time [before and after] for the EDDY")
for subjFolder in subjFolders:
    for time in ['before', 'after']:
        ind_to_write = 1
        with open(os.path.join(preproc_path, subjFolder, f'{time}_index.txt'), 'w') as f:
            for i in range(expectedVolums['AP']):   
                if i > 0 and i in expectedB0s_indxs[f'AP_{time}']:
                    ind_to_write += 1
                f.write("%s\n" %ind_to_write)
print('>> COMPLETED.')


>> create an index.txt file for each subject for each time [before and after] for the EDDY
>> COMPLETED.


#### Run EDDY
* Correct for eddy currents and subject movement (and taking to account the suceptibility field calculated by TOPUP)

In [17]:
print('> Run EDDY using cheshire server GPU (one per subject per time [''before'' and ''after''])')
# files_to_run_each_time = 200
# counter=1
for subjFolder in subjFolders:
    # if int(subjFolder.split('-')[1])<265:
    #     continue

    # # Stop executing more:
    # if counter > files_to_run_each_time:
    #     break

    scansBaseNames = createSubjectScansBaseNames(subjFolder, data_path)

    # check if all expected output files exist:
    for time in ['before', 'after']:
        
        eddy_output_files = glob.glob(os.path.join(preproc_path, subjFolder, f'eddy_unwarped_images_{subjFolder}_{time}*')) # get files in a directory
        expected_EDDY_out_files_exist = len(list(set(eddy_output_files))) >= n_expected_EDDY_output_files # check that the number of unique EDDY files is as expected

        # Run the launch file if not all output files present:
        if not expected_EDDY_out_files_exist:
            eddy_command_to_run = f"{ssh_command_for_cheshire_server} {EDDY_command} --imain={scansBaseNames[f'AP_{time}']}.nii.gz \\\n\
                --mask={os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_iout_avg_brain_mask \\\n\
                --index={os.path.join(preproc_path, subjFolder, time + '_index.txt')}\\\n\
                --acqp={os.path.join(preproc_path, subjFolder, subjFolder + '_' + time + '_acqparams.txt')} \\\n\
                --bvecs={scansBaseNames[f'AP_{time}']}.bvec \\\n\
                --bvals={scansBaseNames[f'AP_{time}']}.bval \\\n\
                --topup={os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s \\\n\
                --out={os.path.join(preproc_path, subjFolder,'eddy_unwarped_images_' + subjFolder)}_{time} \\\n\
                --verbose\
            "
            print(f'">> Running this command on cheshire:\n{eddy_command_to_run}\n')
            os.system(eddy_command_to_run)
            # counter+=1

print(f">> COMPLETED.")

> Run EDDY using cheshire server GPU (one per subject per time [before and after])
>> COMPLETED.


Create EDDY launch files [Currently not in use]

In [21]:
# print('> Create the EDDY launch files (one per subject [''before'' and ''after''] together)')
# for subjFolder in subjFolders:

#     scansBaseNames = createSubjectScansBaseNames(subjFolder, data_path)
#     launch_file = os.path.join(launch_files_path, 'EDDY_' + subjFolder + '_launch.txt')

#     if not os.path.isfile(launch_file):
#         with open(launch_file, 'w') as f:
#             for time in ['before', 'after']:
#                 print(f">> This command was written to {launch_file}:\n{EDDY_command} --imain={scansBaseNames[f'AP_{time}']}.nii.gz \\\n\
#                     --mask={os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_iout_avg_brain_mask \\\n\
#                     --index={os.path.join(preproc_path, subjFolder, time + '_index.txt')}\\\n\
#                     --acqp={os.path.join(preproc_path, subjFolder, subjFolder + '_' + time + '_acqparams.txt')} \\\n\
#                     --bvecs={scansBaseNames[f'AP_{time}']}.bvec \\\n\
#                     --bvals={scansBaseNames[f'AP_{time}']}.bval \\\n\
#                     --topup={os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s \\\n\
#                     --out={os.path.join(preproc_path, subjFolder,'eddy_unwarped_images_' + subjFolder)}_{time} \\\n\
#                     --verbose \n\
#                 ")
#                 #f.write(f"{EDDY_command} --imain={scansBaseNames[f'AP_{time}']}.nii.gz --mask={os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_iout_avg_brain_mask --index={os.path.join(preproc_path, subjFolder, time + '_index.txt')} --acqp={os.path.join(preproc_path, subjFolder, subjFolder + '_' + time + '_acqparams.txt')} --bvecs={scansBaseNames[f'AP_{time}']}.bvec --bvals={scansBaseNames[f'AP_{time}']}.bval --topup={os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s --out={os.path.join(preproc_path, subjFolder,'eddy_unwarped_images_' + subjFolder)}_{time} --verbose\n")
# print(f">> COMPLETED.")

Run EDDY launch files  [Currently not in use]

In [22]:
# print('> Run the EDDY launch files (one per subject [''before'' and ''after''] together)')
# for subjFolder in subjFolders:

#     launch_file = os.path.join(launch_files_path, 'EDDY_' + subjFolder + '_launch.txt')

#     # check if all expected output files exist:
#     expected_EDDY_out_files_exist = []
#     for time in ['before', 'after']:
#         eddy_output_files = glob.glob(os.path.join(preproc_path, subjFolder, f'eddy_unwarped_images_{subjFolder}_{time}*')) # get files in a directory
#         expected_EDDY_out_files_exist.append(len(list(set(eddy_output_files))) == n_expected_EDDY_output_files) # check that the number of unique EDDY files is as expected

#     # Run the launch file if not all output files present:
#     if not all(expected_EDDY_out_files_exist):
#                 print(f">> Running: launch -s {launch_file} -j schonberglab -p {n_cores_EDDY} -r inf")
#                 #os.system(f">> Running: launch -s {launch_file} -j schonberglab -p {n_cores_EDDY} -r inf")
# print(f">> COMPLETED.")

#### Run EDDY QC

In [23]:
for subjFolder in subjFolders:
    # get the sub_B0s_files for the current subject:
    scansBaseNames = createSubjectScansBaseNames(subjFolder, data_path)

    for time in ['before', 'after']:
        if not os.path.isdir(os.path.join(preproc_path, subjFolder, f'eddy_unwarped_images_{subjFolder}_{time}.qc')):
            print(f"eddy_quad {os.path.join(preproc_path, subjFolder, f'eddy_unwarped_images_{subjFolder}_{time}')} \\\n\
                -idx {os.path.join(preproc_path, subjFolder, f'{time}_index.txt')} \\\n\
                -par {os.path.join(preproc_path, subjFolder, subjFolder  + f'_{time}_acqparams.txt')} \\\n\
                -m   {os.path.join(preproc_path, subjFolder, f'topup_{subjFolder}_AP_PA_{time}_b0s_iout_avg_brain_mask.nii.gz')} \\\n\
                -b   {scansBaseNames[f'AP_{time}'] + '.bval'}\
                ")
            os.system(f"eddy_quad {os.path.join(preproc_path, subjFolder, f'eddy_unwarped_images_{subjFolder}_{time}')} \\\n\
                -idx {os.path.join(preproc_path, subjFolder, f'{time}_index.txt')} \\\n\
                -par {os.path.join(preproc_path, subjFolder, subjFolder  + f'_{time}_acqparams.txt')} \\\n\
                -m   {os.path.join(preproc_path, subjFolder, f'topup_{subjFolder}_AP_PA_{time}_b0s_iout_avg_brain_mask.nii.gz')} \\\n\
                -b   {scansBaseNames[f'AP_{time}'] + '.bval'}\
                ")





In [374]:
# Run eddiy squad (group eddy QC):
all_qc_dirs = glob.glob(os.path.join(preproc_path, '*', f'eddy_unwarped_images_*_*.qc'))
all_qc_dirs.sort()
#write the list of eddy QC folders into a text file:
with open(os.path.join(preproc_path, 'eddy_QC_folders.txt'), 'w') as f:
    for item in all_qc_dirs:
        f.write("%s\n" % item)

# run eddy quad
os.system(f"eddy_squad {os.path.join(preproc_path, 'eddy_QC_folders.txt')} -o {os.path.join(preproc_path, 'eddy_squad')}")

0

## DTI modelling

#### Create the DTI folders

In [18]:
print('> Create DTI folders')
for subjFolder in subjFolders:
    try:
        os.makedirs(os.path.join(dti_path, subjFolder), exist_ok=False)
        print('>> Created folder: ' + os.path.join(dti_path, subjFolder))
    except:
        pass

> Create DTI folders


In [19]:
print('> Run DTIFIT (one per subject per time [''before'' and ''after''])')
# files_to_run_each_time = 200
# counter=1
for subjFolder in subjFolders:

    scansBaseNames = createSubjectScansBaseNames(subjFolder, data_path)

    # check if all expected output files exist:
    for time in ['before', 'after']:
        
        dtifit_output_files = glob.glob(os.path.join(dti_path, subjFolder, f'dti_{subjFolder}_{time}*')) # get files in a directory
        expected_DTIFIT_out_files_exist = len(list(set(dtifit_output_files))) >= n_expected_DTIFIT_output_files # check that the number of unique EDDY files is as expected

        # Run the launch file if not all output files present:
        if not expected_DTIFIT_out_files_exist:
            dtifit_command = f"dtifit --data={os.path.join(preproc_path, subjFolder, f'eddy_unwarped_images_{subjFolder}_{time}')} \\\n\
                        --mask={os.path.join(preproc_path, subjFolder, f'topup_{subjFolder}_AP_PA_{time}_b0s_iout_avg_brain_mask')} \\\n\
                        --bvecs={scansBaseNames[f'AP_{time}'] + '.bvec'} \\\n\
                        --bvals={scansBaseNames[f'AP_{time}'] + '.bval'} \\\n\
                        --out={os.path.join(dti_path, subjFolder, f'dti_{subjFolder}_{time}')}"

            print(dtifit_command)
            os.system(dtifit_command)



> Run DTIFIT (one per subject per time [before and after])


# *-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

### Gather subject folders again based on the dti folders!
***** Make sure that before that you ran DWI_pipeline_noTOPUPpipeline.ipynb to include subject without TOPUP *****

In [8]:
print('>> Get sub folders from the DTI map folder')
subjFoldersDTI = [el for el in os.listdir(dti_path) if 'sub' in el]
subjFoldersDTI.sort()

>> Get sub folders from the DTI map folder


In [9]:
len(subjFoldersDTI)

121

# *-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

## Registration to MNI space

#### Create folders for the preprocessed anatomical data (from fMRIprep)

In [11]:
print('> Create registration folders')
for subjFolder in subjFoldersDTI:
    try:
        os.makedirs(os.path.join(dti_path, subjFolder, 'reg'), exist_ok=False)
        print('>> Created folder: ' + os.path.join(dti_path, subjFolder, 'reg'))
    except:
        pass

> Create registration folders


In [12]:
print('> Copy relevant files to the registration folders (T1 processed, related mask, and WM segmentation file')
for subjFolder in subjFoldersDTI:
    anatFilesToCopy = [
        os.path.join(fmriPrepAnatomyDerivatives_path, subjFolder, 'anat', f'{subjFolder}_desc-preproc_T1w.nii.gz'), 
        os.path.join(fmriPrepAnatomyDerivatives_path, subjFolder, 'anat', f'{subjFolder}_desc-brain_mask.nii.gz'), 
        os.path.join(fmriPrepAnatomyDerivatives_path, subjFolder, 'anat', f'{subjFolder}_label-WM_probseg.nii.gz')
    ]
    
    for file in anatFilesToCopy:
        if not os.path.exists(os.path.join(dti_path, subjFolder, 'reg', os.path.split(file)[-1])):
            print(f"cp {file} {os.path.join(dti_path, subjFolder, 'reg')}")
            os.system(f"cp {file} {os.path.join(dti_path, subjFolder, 'reg')}")
    

> Copy relevant files to the registration folders (T1 processed, related mask, and WM segmentation file


In [13]:
print('> Extract the T1w brain')
for subjFolder in subjFoldersDTI:
    sub_T1 = f"{os.path.join(dti_path, subjFolder, 'reg', f'{subjFolder}_desc-preproc_T1w.nii.gz')}"
    sub_T1_mask = f"{os.path.join(dti_path, subjFolder, 'reg', f'{subjFolder}_desc-brain_mask.nii.gz')}"
    extracted_brain = f"{os.path.join(dti_path, subjFolder, 'reg', f'{subjFolder}_desc-preproc_T1w_brain.nii.gz')}"
    if not os.path.exists(extracted_brain):
        print(f"fslmaths {sub_T1} -mul {sub_T1_mask} {extracted_brain}")
        os.system(f"fslmaths {sub_T1} -mul {sub_T1_mask} {extracted_brain}")
print(f">> COMPLETED.")


> Extract the T1w brain
>> COMPLETED.


In [14]:
print('> Extract the first (B0 volume form the eddy_output to use for the native space registration (this volume is the reference volume for the motion correction and thus everything is aligned to it.')
for subjFolder in subjFoldersDTI:
    for time in ['before', 'after']:
        fileToCreate = f"{subjFolder}_first_processed_B0_{time}.nii.gz"
        if not os.path.exists(os.path.join(dti_path, subjFolder, 'reg', fileToCreate)):
            print(f"fslroi {os.path.join(preproc_path, subjFolder, f'eddy_unwarped_images_{subjFolder}_{time}.nii.gz')} {os.path.join(dti_path, subjFolder, 'reg', fileToCreate)} 0 1")
            os.system(f"fslroi {os.path.join(preproc_path, subjFolder, f'eddy_unwarped_images_{subjFolder}_{time}.nii.gz')} {os.path.join(dti_path, subjFolder, 'reg', fileToCreate)} 0 1")

print(f">> COMPLETED.")

> Extract the first (B0 volume form the eddy_output to use for the native space registration (this volume is the reference volume for the motion correction and thus everything is aligned to it.
>> COMPLETED.


#### Register the first (processed) B0 to the preprocessed T1w in native space (using epi_reg)

This includes:
1) using epi_reg on a B0 imgae to the native T1 space.
2) register the T1w to MNI [by running flirt and then fnirt]. * note that fnirt runs on the secnd cell as flirt need to be completed first

In [15]:
print('> Execute EPI_REG (create launch files and run them)')
for subjFolder in subjFoldersDTI:
    for time in ['before', 'after']:
        if not os.path.exists(os.path.join(dti_path, subjFolder, 'reg', f'{subjFolder}_first_processed_B0_reg_to_native_T1_{time}.nii.gz')):
            launch_file_EPI_TO_REG = os.path.join(launch_files_path, 'EPI_REG_' + subjFolder + '_' + time + '_launch.txt')
            print(
                f"epi_reg -v --epi={os.path.join(dti_path, subjFolder, 'reg', f'{subjFolder}_first_processed_B0_{time}.nii.gz')} \\\n\
                --t1={os.path.join(dti_path, subjFolder, 'reg', f'{subjFolder}_desc-preproc_T1w.nii.gz')} \\\n\
                --t1brain={os.path.join(dti_path, subjFolder, 'reg', f'{subjFolder}_desc-preproc_T1w_brain.nii.gz')} \\\n\
                --out={os.path.join(dti_path, subjFolder, 'reg', f'{subjFolder}_first_processed_B0_reg_to_native_T1_{time}')}\n"
                #I removed this from the command (letting it create WM_seg on its own): --wmseg={os.path.join(dti_path, subjFolder, 'reg', f'{subjFolder}_label-WM_probseg.nii.gz')} \\\n\
            )
            epi_reg_command_to_run = f"epi_reg -v --epi={os.path.join(dti_path, subjFolder, 'reg', f'{subjFolder}_first_processed_B0_{time}.nii.gz')} --t1={os.path.join(dti_path, subjFolder, 'reg', f'{subjFolder}_desc-preproc_T1w.nii.gz')} --t1brain={os.path.join(dti_path, subjFolder, 'reg', f'{subjFolder}_desc-preproc_T1w_brain.nii.gz')} --out={os.path.join(dti_path, subjFolder, 'reg', f'{subjFolder}_first_processed_B0_reg_to_native_T1_{time}')}\n"
            #I removed this from the command (letting it create WM_seg on its own): --wmseg={os.path.join(dti_path, subjFolder, 'reg', f'{subjFolder}_label-WM_probseg.nii.gz')} \\\n\
    
            with open(launch_file_EPI_TO_REG, 'w') as f:
                 f.write(epi_reg_command_to_run)
            os.system(f"launch -s {launch_file_EPI_TO_REG} -j schonberglab -p {n_cores_TOPUP} -r inf")    

print(f">> COMPLETED.")



> Execute EPI_REG (create launch files and run them)
>> COMPLETED.


#### Apply all transofrmation to take all the relevant maps to a common space (MNI)

In [18]:
c3d_affine_tool_command = '/export2/DATA/HIS/HIS_server/norm_tools/c3d_affine_tool'
antsApplyTransforms_command = '/export2/DATA/HIS/HIS_server/norm_tools/antsApplyTransforms'
bids_deriv_path = '/export2/DATA/HIS/HIS_server/BIDS/derivatives/fmriprep'

In [19]:
print('> Apply the transformation on the MD and FA maps')
for subjFolder in subjFoldersDTI:
    for time in ['before', 'after']:
        if not os.path.exists(f"{os.path.join(dti_path, subjFolder, 'reg', f'{subjFolder}_first_processed_B0_reg_to_native_T1_{time}_ANTs-format.txt')}"):
            print(f"{c3d_affine_tool_command} -ref {os.path.join(dti_path, subjFolder, 'reg', f'{subjFolder}_desc-preproc_T1w.nii.gz')} -src {os.path.join(dti_path, subjFolder, 'reg', f'{subjFolder}_first_processed_B0_{time}.nii.gz')} {os.path.join(dti_path, subjFolder, 'reg', f'{subjFolder}_first_processed_B0_reg_to_native_T1_{time}.mat')} -fsl2ras -oitk {os.path.join(dti_path, subjFolder, 'reg', f'{subjFolder}_first_processed_B0_reg_to_native_T1_{time}_ANTs-format.txt')}")
            os.system(f"{c3d_affine_tool_command} -ref {os.path.join(dti_path, subjFolder, 'reg', f'{subjFolder}_desc-preproc_T1w.nii.gz')} -src {os.path.join(dti_path, subjFolder, 'reg', f'{subjFolder}_first_processed_B0_{time}.nii.gz')} {os.path.join(dti_path, subjFolder, 'reg', f'{subjFolder}_first_processed_B0_reg_to_native_T1_{time}.mat')} -fsl2ras -oitk {os.path.join(dti_path, subjFolder, 'reg', f'{subjFolder}_first_processed_B0_reg_to_native_T1_{time}_ANTs-format.txt')}")
print('> COMPLETED')

> Apply the transformation on the MD and FA maps
> COMPLETED


In [24]:
print('> Apply the transformation on the MD and FA maps')
for subjFolder in subjFoldersDTI:
    for time in ['before', 'after']:
        if not os.path.exists(os.path.join(stats_path, 'MD', f'{subjFolder}_in-MNI_MD_{time}.nii.gz')) \
        or not os.path.exists(os.path.join(stats_path, 'FA', f'{subjFolder}_in-MNI_FA_{time}.nii.gz')):
            for map_type in ['MD','FA']:
                print(f"{antsApplyTransforms_command} -i {os.path.join(dti_path, subjFolder, f'dti_{subjFolder}_{time}_{map_type}.nii.gz')} -r ${{FSLDIR}}/data/standard/MNI152_T1_2mm.nii.gz -o {os.path.join(stats_path, map_type, f'{subjFolder}_in-MNI_{map_type}_{time}.nii.gz')} -t {os.path.join(bids_deriv_path, subjFolder, 'anat', f'{subjFolder}_from-T1w_to-MNI152NLin2009cAsym_mode-image_xfm.h5')} -t {os.path.join(dti_path, subjFolder, 'reg', f'{subjFolder}_first_processed_B0_reg_to_native_T1_{time}_ANTs-format.txt')}")
                os.system(f"{antsApplyTransforms_command} -i {os.path.join(dti_path, subjFolder, f'dti_{subjFolder}_{time}_{map_type}.nii.gz')} -r ${{FSLDIR}}/data/standard/MNI152_T1_2mm.nii.gz -o {os.path.join(stats_path, map_type, f'{subjFolder}_in-MNI_{map_type}_{time}.nii.gz')} -t {os.path.join(bids_deriv_path, subjFolder, 'anat', f'{subjFolder}_from-T1w_to-MNI152NLin2009cAsym_mode-image_xfm.h5')} -t {os.path.join(dti_path, subjFolder, 'reg', f'{subjFolder}_first_processed_B0_reg_to_native_T1_{time}_ANTs-format.txt')}")
print('> COMPLETED')

> Apply the transformation on the MD and FA maps
> COMPLETED


# *-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

# Spatial smoothing

Spatial smoothing with 6-mm FWHM Gaussian kernels is carried out using:

fslmaths input -s 2.547965 output [6mm FWHM]

See https://brainder.org/2011/08/20/gaussian-kernels-convert-fwhm-to-sigma/ for the conversion formula between the gaussian sigma (taken as the smoothing parameter by fslmaths) and FWHM.

In [31]:
print('> Apply the spatial smoothing on the MD and FA maps')
for subjFolder in subjFoldersDTI:
    for time in ['before', 'after']:
        if not os.path.exists(os.path.join(stats_path, 'MD', f'{subjFolder}_in-MNI_ss6mm_MD_{time}.nii.gz')) \
        or not os.path.exists(os.path.join(stats_path, 'FA', f'{subjFolder}_in-MNI_ss6mm_FA_{time}.nii.gz')):
            for map_type in ['MD','FA']:
                print(f"fslmaths {os.path.join(stats_path, map_type, f'{subjFolder}_in-MNI_{map_type}_{time}.nii.gz')} -s 2.547965 {os.path.join(stats_path, map_type, f'{subjFolder}_in-MNI_ss6mm_{map_type}_{time}.nii.gz')}")
                os.system(f"fslmaths {os.path.join(stats_path, map_type, f'{subjFolder}_in-MNI_{map_type}_{time}.nii.gz')} -s 2.547965 {os.path.join(stats_path, map_type, f'{subjFolder}_in-MNI_ss6mm_{map_type}_{time}.nii.gz')}")
print('> COMPLETED')

> Apply the spatial smoothing on the MD and FA maps
fslmaths /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/MD/sub-103_in-MNI_MD_before.nii.gz -s 2.547965 /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/MD/sub-103_in-MNI_ss6mm_MD_before.nii.gz
fslmaths /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/FA/sub-103_in-MNI_FA_before.nii.gz -s 2.547965 /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/FA/sub-103_in-MNI_ss6mm_FA_before.nii.gz
fslmaths /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/MD/sub-103_in-MNI_MD_after.nii.gz -s 2.547965 /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/MD/sub-103_in-MNI_ss6mm_MD_after.nii.gz
fslmaths /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/FA/sub-103_in-MNI_FA_after.nii.gz -s 2.547965 /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/FA/sub-103_in-MNI_ss6mm_FA_after.nii.gz
fslmaths /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/MD/sub

# *-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

# Statistical Analysis: BETWEEN-GRUOP

The analysis is based on this page: https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/GLM#ANOVA:_2-groups.2C_2-levels_per_subject_.282-way_Mixed_Effect_ANOVA.29
Specifically:
"Randomise details
Due to how the data would need to be permuted, the FEAT model may not be used in randomise. Instead, just as in the paired t-test example, paired differences within-subject would be computed via fslmaths and a two-sample t-test could be used to test whether the run1-run2 difference differed between the two groups."

Thus the first step (calculating the differenceswithin-subject) is as here: https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/GLM#Single-Group_Paired_Difference_.28Paired_T-Test.29
And the second is as here 


In [ ]:
print('> Calculate the difference of AFTER minus BEFORE and BEFORE minus AFTER for the MD and FA maps')
for subjFolder in subjFoldersDTI:
    if not os.path.exists(os.path.join(stats_path, 'MD', f'{subjFolder}_in-MNI_ss6mm_MD_AFTER-minus-BEFORE.nii.gz')) \
    or not os.path.exists(os.path.join(stats_path, 'FA', f'{subjFolder}_in-MNI_ss6mm_FA_AFTER-minus-BEFORE.nii.gz')) \
    or not os.path.exists(os.path.join(stats_path, 'MD', f'{subjFolder}_in-MNI_ss6mm_MD_BEFORE-minus-AFTER.nii.gz')) \
    or not os.path.exists(os.path.join(stats_path, 'FA', f'{subjFolder}_in-MNI_ss6mm_FA_BEFORE-minus-AFTER.nii.gz')):      
     
        for map_type in ['MD','FA']:          
            substraction_command = f"fslmaths {os.path.join(stats_path, map_type, f'{subjFolder}_in-MNI_ss6mm_{map_type}_after.nii.gz')} -sub {os.path.join(stats_path, map_type, f'{subjFolder}_in-MNI_ss6mm_{map_type}_before.nii.gz')} {os.path.join(stats_path, map_type, f'{subjFolder}_in-MNI_{map_type}_AFTER-minus-BEFORE.nii.gz')}"
            print(substraction_command)
            os.system(substraction_command)    
            substraction_command2 = f"fslmaths {os.path.join(stats_path, map_type, f'{subjFolder}_in-MNI_ss6mm_{map_type}_before.nii.gz')} -sub {os.path.join(stats_path, map_type, f'{subjFolder}_in-MNI_ss6mm_{map_type}_after.nii.gz')} {os.path.join(stats_path, map_type, f'{subjFolder}_in-MNI_{map_type}_BEFORE-minus-AFTER.nii.gz')}"
            print(substraction_command2)
            os.system(substraction_command2)    

In [33]:
print('> Create one file with all subjects of AFTER minus BEFORE and one with all Before minus AFTER for the MD and FA maps')
for map_type in ['MD','FA']:          
    print(f"fslmerge -t {os.path.join(stats_path, map_type, f'ALL_SUBJECTS_AFTER_minus_BEFORE_{map_type}')} `ls {os.path.join(stats_path, map_type, f'*in-MNI_{map_type}_AFTER-minus-BEFORE.nii.gz')}`")
    os.system(f"fslmerge -t {os.path.join(stats_path, map_type, f'ALL_SUBJECTS_AFTER_minus_BEFORE_{map_type}')} `ls {os.path.join(stats_path, map_type, f'*in-MNI_{map_type}_AFTER-minus-BEFORE.nii.gz')}`")
    print(f"fslmerge -t {os.path.join(stats_path, map_type, f'ALL_SUBJECTS_BEFORE_minus_AFTER_{map_type}')} `ls {os.path.join(stats_path, map_type, f'*in-MNI_{map_type}_BEFORE-minus-AFTER.nii.gz')}`")
    os.system(f"fslmerge -t {os.path.join(stats_path, map_type, f'ALL_SUBJECTS_BEFORE_minus_AFTER_{map_type}')} `ls {os.path.join(stats_path, map_type, f'*in-MNI_{map_type}_BEFORE-minus-AFTER.nii.gz')}`")

> Create one file with all subjects of AFTER minus BEFORE and one withh all Before minus AFTER for the MD and FA maps
fslmerge -t /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/MD/ALL_SUBJECTS_AFTER_minus_BEFORE_MD `ls /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/MD/*in-MNI_MD_AFTER-minus-BEFORE.nii.gz`
fslmerge -t /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/MD/ALL_SUBJECTS_BEFORE_minus_AFTER_MD `ls /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/MD/*in-MNI_MD_BEFORE-minus-AFTER.nii.gz`
fslmerge -t /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/FA/ALL_SUBJECTS_AFTER_minus_BEFORE_FA `ls /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/FA/*in-MNI_FA_AFTER-minus-BEFORE.nii.gz`
fslmerge -t /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/FA/ALL_SUBJECTS_BEFORE_minus_AFTER_FA `ls /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/FA/*in-MNI_FA_BEFORE-minus-AFTER.nii.gz`


In [34]:
print('> Create the between group design matrix (of the calculated differences) - same is unpaired t-test (on the after-before differences)')
# create the folder
try:
    os.makedirs(os.path.join(models_path, 'between_groups'), exist_ok=False)
    print('>> Created folder: ' + os.path.join(models_path, 'between_groups'))
except:
    pass

#create the design matrix and contrasts:
N_1_day_group = len(set([sub for sub in subjFoldersDTI if int(sub.split('-')[-1])<200]))
N_3_day_group = len(set([sub for sub in subjFoldersDTI if int(sub.split('-')[-1])>200]))

print(f"design_ttest2 {os.path.join(models_path, 'between_groups','design_unpaired_ttest')} {N_1_day_group} {N_3_day_group}")
os.system(f"design_ttest2 {os.path.join(models_path, 'between_groups','design_unpaired_ttest')} {N_1_day_group} {N_3_day_group}")

> Create the between group design matrix (of the calculated differences) - same is unpaired t-test (on the after-before differences)
design_ttest2 /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/models/between_groups/design_unpaired_ttest 61 60


0

### Settings

In [10]:
# a function to run randomise for the between-group analyses:
def runRandomiseGroupAnalysis(map_type, region):
    print(f'Running randomise for between-groups analysis:\n--- {map_type} | {region} ---\n')
    #print(f'MAKE SURE TO REPLACE IN THE FUNCTION THE the print to os.system before running the command !!!\n')
    os.system(f"randomise_parallel \
        -i {os.path.join(stats_path, map_type, f'ALL_SUBJECTS_AFTER_minus_BEFORE_{map_type}')} \
        -o {os.path.join(stats_path, map_type, f'ALL_SUBJECTS_AFTER_minus_BEFORE_{map_type}_{region}')} \
        {'' if region=='whole_brain' else f'-m {masks_paths[region]}'} \
        -d {os.path.join(stats_path, 'models', 'between_groups', f'design_unpaired_ttest.mat')} \
        -t {os.path.join(stats_path, 'models', 'between_groups', f'design_unpaired_ttest.con')}  \
        -n 6000 -C 3.1")

# Get cluster info:
def getClusterInfoGroupAnalysis(map_type, region, ReversedSigThresh='0.90'):
    print(f'Testing stats for between-group analysis:\n--- {map_type} | {region} ---\n')
    print(f'Test 1-Day > 3-Day\n')
    # * Increase in FA in the 1-day relatvie to the 3-days (or a decrease in the 3-day...)
    print(os.popen(f'cluster \
        -i {os.path.join(stats_path, map_type, f"ALL_SUBJECTS_AFTER_minus_BEFORE_{map_type}_{region}_clusterm_corrp_tstat1.nii.gz")} \
        -c {os.path.join(stats_path, map_type, f"ALL_SUBJECTS_AFTER_minus_BEFORE_{map_type}_{region}_tstat1.nii.gz")}\
        -t {ReversedSigThresh} \
        --scalarname=corrp' + '"1-p"').read())

    print(f'3-Day > 1-Day\n')
    # * Increase in FA in the 3-day relatvie to the 1-days (or a decrease in the 1-day...)
    print(os.popen(f'cluster \
        -i {os.path.join(stats_path, map_type, f"ALL_SUBJECTS_AFTER_minus_BEFORE_{map_type}_{region}_clusterm_corrp_tstat2.nii.gz")} \
        -c {os.path.join(stats_path, map_type, f"ALL_SUBJECTS_AFTER_minus_BEFORE_{map_type}_{region}_tstat2.nii.gz")}\
        -t {ReversedSigThresh} \
        --scalarname=corrp' + '"1-p"').read())


## Run the analyses analyses (through the terminal)

### Whole brain - FA

In [18]:
map_type='FA'
region='whole_brain'

In [38]:
runRandomiseGroupAnalysis(map_type, region)

Running randomise for between-groups analysis:
--- FA | whole_brain ---



### Putamen - FA

In [20]:
region='Putamen'

In [41]:
runRandomiseGroupAnalysis(map_type, region)

Running randomise for between-groups analysis:
--- FA | Putamen ---



### Caudate - FA

In [22]:
region='Caudate'

In [44]:
runRandomiseGroupAnalysis(map_type, region)

Running randomise for between-groups analysis:
--- FA | Caudate ---



### vmPFC - FA

In [24]:
region='vmPFC'

In [47]:
runRandomiseGroupAnalysis(map_type, region)

Running randomise for between-groups analysis:
--- FA | vmPFC ---



### Whole brain - MD

In [26]:
map_type='MD'
region='whole_brain'

In [50]:
runRandomiseGroupAnalysis(map_type, region)

Running randomise for between-groups analysis:
--- MD | whole_brain ---



### Putamen - MD

In [28]:
region='Putamen'

In [53]:
runRandomiseGroupAnalysis(map_type, region)

Running randomise for between-groups analysis:
--- MD | Putamen ---



### Caudate - MD

In [30]:
region='Caudate'

In [56]:
runRandomiseGroupAnalysis(map_type, region)

Running randomise for between-groups analysis:
--- MD | Caudate ---



### vmPFC - MD

In [32]:
region='vmPFC'

In [59]:
runRandomiseGroupAnalysis(map_type, region)

Running randomise for between-groups analysis:
--- MD | vmPFC ---



## SUMMARY:

In [12]:
# get the cluster info for the between-group analysis:
for map_type in ['FA', 'MD']:
    for region in ['whole_brain', 'Putamen', 'Caudate', 'vmPFC']:
        getClusterInfoGroupAnalysis(map_type, region, ReversedSigThresh='0.90')

Testing stats for between-group analysis:
--- FA | whole_brain ---

Test 1-Day > 3-Day

Cluster Index	Voxels	corrp1-p-MAX	corrp1-p-MAX X (vox)	corrp1-p-MAX Y (vox)	corrp1-p-MAX Z (vox)	corrp1-p-COG X (vox)	corrp1-p-COG Y (vox)	corrp1-p-COG Z (vox)	COPE-MAX	COPE-MAX X (vox)	COPE-MAX Y (vox)	COPE-MAX Z (vox)	COPE-MEAN
2	76	0.755	76	58	12	75.8	59.8	13.9	5.44	76	60	14	4.02
1	72	0.624	37	49	58	37.8	52.1	61.3	4.05	37	50	59	3.39

3-Day > 1-Day

Cluster Index	Voxels	corrp1-p-MAX	corrp1-p-MAX X (vox)	corrp1-p-MAX Y (vox)	corrp1-p-MAX Z (vox)	corrp1-p-COG X (vox)	corrp1-p-COG Y (vox)	corrp1-p-COG Z (vox)	COPE-MAX	COPE-MAX X (vox)	COPE-MAX Y (vox)	COPE-MAX Z (vox)	COPE-MEAN

Testing stats for between-group analysis:
--- FA | Putamen ---

Test 1-Day > 3-Day

Cluster Index	Voxels	corrp1-p-MAX	corrp1-p-MAX X (vox)	corrp1-p-MAX Y (vox)	corrp1-p-MAX Z (vox)	corrp1-p-COG X (vox)	corrp1-p-COG Y (vox)	corrp1-p-COG Z (vox)	COPE-MAX	COPE-MAX X (vox)	COPE-MAX Y (vox)	COPE-MAX Z (vox)	COPE-MEAN

3-Day > 1-Da

# *-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

# Statistics - individual differences

### Create the habit index csv file (by running a dedicated matlab folder)


In [25]:
# Create the habit index csv files (by running a dedicated matlab folder)
!matlab -nodesktop -nosplash -r "run('/export2/DATA/HIS/HIS_server/codes/create_HABIT_INDEX_table')";

MATLAB is selecting SOFTWARE OPENGL rendering.
=
                            < M A T L A B (R) >
                  Copyright 1984-2020 The MathWorks, Inc.
                  R2020a (9.8.0.1323502) 64-bit (glnxa64)
                             February 25, 2020

 
To get started, type doc.
For product information, visit www.mathworks.com.
 
** CREATED new habit index file **
>

### Prepare and arrange the behavioral data and DTI folder data for all together and for each group seperately

In [16]:
# First, create subjects list folder for each group:
subjFoldersDTI_short = [sub for sub in subjFoldersDTI if int(sub.split('-')[-1])<200]
subjFoldersDTI_long = [sub for sub in subjFoldersDTI if int(sub.split('-')[-1])>200]

# Get the behavioral data:
behav_all = pd.read_csv(os.path.join(behav_data_path, 'habitIndex_ALL.csv'))
# remove subjects that don't have DTI data
absent_from_DTI_data = [sub for sub in behav_all.subID if 'sub-' + str(sub) not in subjFoldersDTI]
behav_all = behav_all[~behav_all.subID.isin(absent_from_DTI_data)]
# now make sure that subject lists are matching:
if not ['sub-' + str(sub) for sub in behav_all.subID] == subjFoldersDTI:
    raise Exception('Subject lists are not matching')

# Repeat for the long training groups:
behav_long = pd.read_csv(os.path.join(behav_data_path, 'habitIndex_LONG.csv'))
# remove subjects that don't have DTI data
absent_from_DTI_data = [sub for sub in behav_long.subID if 'sub-' + str(sub) not in subjFoldersDTI_long]
behav_long = behav_long[~behav_long.subID.isin(absent_from_DTI_data)]
# now make sure that subject lists are matching:
if not ['sub-' + str(sub) for sub in behav_long.subID] == subjFoldersDTI_long:
    raise Exception('Subject lists are not matching')
    
# Repeat for the short training groups:
behav_short = pd.read_csv(os.path.join(behav_data_path, 'habitIndex_SHORT.csv'))
# remove subjects that don't have DTI data
absent_from_DTI_data = [sub for sub in behav_short.subID if 'sub-' + str(sub) not in subjFoldersDTI_short]
behav_short = behav_short[~behav_short.subID.isin(absent_from_DTI_data)]
# now make sure that subject lists are matching:
if not ['sub-' + str(sub) for sub in behav_short.subID] == subjFoldersDTI_short:
    raise Exception('Subject lists are not matching')



### prepare the design matrix

In [27]:
print('> Create the individual differences design matrix')
# define file names:
design_indiv_diff_LONG = os.path.join(models_path, 'individual_differences', 'design_indivdual_diff_LONG.mat')
design_indiv_diff_SHORT = os.path.join(models_path, 'individual_differences', 'design_indivdual_diff_SHORT.mat')
contrasts_indiv_diff = os.path.join(models_path, 'individual_differences', 'design_indivdual_diff.con')

# create the folder
try:
    os.makedirs(os.path.join(models_path, 'individual_differences'), exist_ok=False)
    print('>> Created folder: ' + os.path.join(models_path, 'individual_differences'))
except:
    pass

# write the design matrix files:
behav_long.habit_index_mean_centered.to_csv(f"{design_indiv_diff_LONG.split('.')[0]}_temp.mat", index=False, header=False)
behav_short.habit_index_mean_centered.to_csv(f"{design_indiv_diff_SHORT.split('.')[0]}_temp.mat", index=False, header=False)

# write the contrast file:
with open(f"{contrasts_indiv_diff.split('.')[0]}_temp.mat", 'w') as f:
    f.write('1\n-1\n')

# use the fsl command Text2Vect to make the design files good for randomise:
print(f"Text2Vest {design_indiv_diff_LONG.split('.')[0]}_temp.mat {design_indiv_diff_LONG}")
os.system(f"Text2Vest {design_indiv_diff_LONG.split('.')[0]}_temp.mat {design_indiv_diff_LONG}")
print(f"Text2Vest {design_indiv_diff_SHORT.split('.')[0]}_temp.mat {design_indiv_diff_SHORT}")
os.system(f"Text2Vest {design_indiv_diff_SHORT.split('.')[0]}_temp.mat {design_indiv_diff_SHORT}")
print(f"Text2Vest {contrasts_indiv_diff.split('.')[0]}_temp.mat {contrasts_indiv_diff}")
os.system(f"Text2Vest {contrasts_indiv_diff.split('.')[0]}_temp.mat {contrasts_indiv_diff}")

# finally, remove the temporary files:
os.remove(f"{design_indiv_diff_LONG.split('.')[0]}_temp.mat")
os.remove(f"{design_indiv_diff_SHORT.split('.')[0]}_temp.mat")
os.remove(f"{contrasts_indiv_diff.split('.')[0]}_temp.mat")

> Create the individual differences design matrix
Text2Vest /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/models/individual_differences/design_indivdual_diff_LONG_temp.mat /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/models/individual_differences/design_indivdual_diff_LONG.mat
Text2Vest /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/models/individual_differences/design_indivdual_diff_SHORT_temp.mat /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/models/individual_differences/design_indivdual_diff_SHORT.mat
Text2Vest /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/models/individual_differences/design_indivdual_diff_temp.mat /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/models/individual_differences/design_indivdual_diff.con


### Create one file with each group's participants together

In [ ]:
for map_type in ['MD','FA']:          
    print(f"fslmerge -t {os.path.join(stats_path, map_type, f'LONG_individualDiff_AFTER_minus_BEFORE_{map_type}')} {' '.join([os.path.join(stats_path, map_type, f'{sub}_in-MNI_{map_type}_AFTER-minus-BEFORE.nii.gz') for sub in subjFoldersDTI_long])}")
    os.system(f"fslmerge -t {os.path.join(stats_path, map_type, f'LONG_individualDiff_AFTER_minus_BEFORE_{map_type}')} {' '.join([os.path.join(stats_path, map_type, f'{sub}_in-MNI_{map_type}_AFTER-minus-BEFORE.nii.gz') for sub in subjFoldersDTI_long])}")

    print(f"fslmerge -t {os.path.join(stats_path, map_type, f'SHORT_individualDiff_AFTER_minus_BEFORE_{map_type}')} {' '.join([os.path.join(stats_path, map_type, f'{sub}_in-MNI_{map_type}_AFTER-minus-BEFORE.nii.gz') for sub in subjFoldersDTI_short])}")
    os.system(f"fslmerge -t {os.path.join(stats_path, map_type, f'SHORT_individualDiff_AFTER_minus_BEFORE_{map_type}')} {' '.join([os.path.join(stats_path, map_type, f'{sub}_in-MNI_{map_type}_AFTER-minus-BEFORE.nii.gz') for sub in subjFoldersDTI_short])}")

    print(f"fslmerge -t {os.path.join(stats_path, map_type, f'LONG_individualDiff_BEFORE_minus_AFTER_{map_type}')} {' '.join([os.path.join(stats_path, map_type, f'{sub}_in-MNI_{map_type}_BEFORE-minus-AFTER.nii.gz') for sub in subjFoldersDTI_long])}")
    os.system(f"fslmerge -t {os.path.join(stats_path, map_type, f'LONG_individualDiff_BEFORE_minus_AFTER_{map_type}')} {' '.join([os.path.join(stats_path, map_type, f'{sub}_in-MNI_{map_type}_BEFORE-minus-AFTER.nii.gz') for sub in subjFoldersDTI_long])}")

    print(f"fslmerge -t {os.path.join(stats_path, map_type, f'SHORT_individualDiff_BEFORE_minus_AFTER_{map_type}')} {' '.join([os.path.join(stats_path, map_type, f'{sub}_in-MNI_{map_type}_BEFORE-minus-AFTER.nii.gz') for sub in subjFoldersDTI_short])}")
    os.system(f"fslmerge -t {os.path.join(stats_path, map_type, f'SHORT_individualDiff_BEFORE_minus_AFTER_{map_type}')} {' '.join([os.path.join(stats_path, map_type, f'{sub}_in-MNI_{map_type}_BEFORE-minus-AFTER.nii.gz') for sub in subjFoldersDTI_short])}")


### Settings

In [17]:
# a function to run randomise for the individual differences analyses:
def runRandomiseIndividualDiffs(group, map_type, region):
    print(f'Running randomise for individual differences:\n--- {group} training group | {map_type} | {region} ---\n')
    #print(f'MAKE SURE TO REPLACE IN THE FUNCTION THE the print to os.system before running the command !!!\n')
    os.system(f"randomise_parallel \
        -i {os.path.join(stats_path, map_type, f'{group}_individualDiff_AFTER_minus_BEFORE_{map_type}')} \
        -o {os.path.join(stats_path, map_type, f'{group}_individualDiff_AFTER_minus_BEFORE_{map_type}_{region}')} \
        {'' if region=='whole_brain' else f'-m {masks_paths[region]}'} \
        -d {os.path.join(stats_path, 'models', 'individual_differences', f'design_indivdual_diff_{group}.mat')} \
        -t {os.path.join(stats_path, 'models', 'individual_differences', 'design_indivdual_diff.con')}  \
        -n 6000 -C 3.1 -D")


# Get cluster info:
def getClusterInfoIndividualDiffs(group, map_type, region, ReversedSigThresh='0.90'):
    print(f'Test for POSITIVE relationship between:\n--- increase in {map_type} and GOAL-DIRECTEDNESS in: {region} | group: {group} ---\n')
    print(os.popen(f'cluster \
        -i {os.path.join(stats_path, map_type, f"{group}_individualDiff_AFTER_minus_BEFORE_{map_type}_{region}_clusterm_corrp_tstat1.nii.gz")} \
        -c {os.path.join(stats_path, map_type, f"{group}_individualDiff_AFTER_minus_BEFORE_{map_type}_{region}_tstat1.nii.gz")}\
        -t {ReversedSigThresh} \
        --scalarname=corrp' + '"1-p"').read())

    print(f'Test for NEGATIVE relationship between:\n--- increase in {map_type} and GOAL-DIRECTEDNESS in: {region} | group: {group} ---\n')
    print(os.popen(f'cluster \
        -i {os.path.join(stats_path, map_type, f"{group}_individualDiff_AFTER_minus_BEFORE_{map_type}_{region}_clusterm_corrp_tstat2.nii.gz")} \
        -c {os.path.join(stats_path, map_type, f"{group}_individualDiff_AFTER_minus_BEFORE_{map_type}_{region}_tstat2.nii.gz")}\
        -t {ReversedSigThresh} \
        --scalarname=corrp' + '"1-p"').read())


## SHORT TRAINING

In [11]:
group = 'SHORT'

### Whole brain - FA

In [12]:
map_type='FA'
region='whole_brain'

In [21]:
runRandomiseIndividualDiffs(group, map_type, region)

Running randomise for individual differences:
--- SHORT training group | FA | whole_brain ---



### Putamen - FA

In [48]:
region='Putamen'

In [49]:
runRandomiseIndividualDiffs(group, map_type, region)

Running randomise for individual differences:
--- SHORT training group | FA | Putamen ---



### Caudate - FA

In [51]:
region='Caudate'

In [53]:
runRandomiseIndividualDiffs(group, map_type, region)

Running randomise for individual differences:
--- SHORT training group | FA | Caudate ---



### vmPFC - FA

In [55]:
region='vmPFC'

In [56]:
runRandomiseIndividualDiffs(group, map_type, region)

Running randomise for individual differences:
--- SHORT training group | FA | vmPFC ---



### Whole brain - MD

In [15]:
map_type='MD'
region='whole_brain'

In [23]:
runRandomiseIndividualDiffs(group, map_type, region)

Running randomise for individual differences:
--- SHORT training group | MD | whole_brain ---



### Putamen - MD

In [25]:
region='Putamen'

In [26]:
runRandomiseIndividualDiffs(group, map_type, region)

Running randomise for individual differences:
--- SHORT training group | MD | Putamen ---



### Caudate - MD

In [28]:
region='Caudate'

In [29]:
runRandomiseIndividualDiffs(group, map_type, region)

Running randomise for individual differences:
--- SHORT training group | MD | Caudate ---



### vmPFC - MD

In [17]:
region='vmPFC'

In [32]:
runRandomiseIndividualDiffs(group, map_type, region)

Running randomise for individual differences:
--- SHORT training group | MD | vmPFC ---



## LONG TRAINING

In [30]:
group = 'LONG'

### Whole brain - FA

In [31]:
map_type='FA'
region='whole_brain'

In [36]:
runRandomiseIndividualDiffs(group, map_type, region)

Running randomise for individual differences:
--- LONG training group | FA | whole_brain ---



### Putamen - FA

In [32]:
region='Putamen'

In [33]:
runRandomiseIndividualDiffs(group, map_type, region)

Running randomise for individual differences:
--- LONG training group | FA | Putamen ---



### Caudate - FA

In [28]:
region='Caudate'

In [42]:
runRandomiseIndividualDiffs(group, map_type, region)

Running randomise for individual differences:
--- LONG training group | FA | Caudate ---



### vmPFC - FA

In [32]:
region='vmPFC'

In [33]:
runRandomiseIndividualDiffs(group, map_type, region)

Running randomise for individual differences:
--- LONG training group | FA | vmPFC ---



### Whole brain - MD

In [13]:
map_type='MD'
region='whole_brain'

In [37]:
runRandomiseIndividualDiffs(group, map_type, region)

Running randomise for individual differences:
--- LONG training group | MD | whole_brain ---



### Putamen - MD

In [39]:
region='Putamen'

In [40]:
runRandomiseIndividualDiffs(group, map_type, region)

Running randomise for individual differences:
--- LONG training group | MD | Putamen ---



### Caudate - MD

In [42]:
region='Caudate'

In [43]:
runRandomiseIndividualDiffs(group, map_type, region)

Running randomise for individual differences:
--- LONG training group | MD | Caudate ---



### vmPFC - MD

In [14]:
region='vmPFC'

In [46]:
runRandomiseIndividualDiffs(group, map_type, region)

Running randomise for individual differences:
--- LONG training group | MD | vmPFC ---



## SUMMARY:

In [60]:
# Get cluster info:
for group in ['SHORT', 'LONG']:
    for map_type in ['FA', 'MD']:
        for region in ['whole_brain', 'Putamen', 'Caudate', 'vmPFC']:
            getClusterInfoIndividualDiffs(group, map_type, region, ReversedSigThresh='0.90')



Test for POSITIVE relationship between:
--- increase in FA and GOAL-DIRECTEDNESS in: whole_brain | group: SHORT ---

Cluster Index	Voxels	corrp1-p-MAX	corrp1-p-MAX X (vox)	corrp1-p-MAX Y (vox)	corrp1-p-MAX Z (vox)	corrp1-p-COG X (vox)	corrp1-p-COG Y (vox)	corrp1-p-COG Z (vox)	COPE-MAX	COPE-MAX X (vox)	COPE-MAX Y (vox)	COPE-MAX Z (vox)	COPE-MEAN

Test for NEGATIVE relationship between:
--- increase in FA and GOAL-DIRECTEDNESS in: whole_brain | group: SHORT ---

Cluster Index	Voxels	corrp1-p-MAX	corrp1-p-MAX X (vox)	corrp1-p-MAX Y (vox)	corrp1-p-MAX Z (vox)	corrp1-p-COG X (vox)	corrp1-p-COG Y (vox)	corrp1-p-COG Z (vox)	COPE-MAX	COPE-MAX X (vox)	COPE-MAX Y (vox)	COPE-MAX Z (vox)	COPE-MEAN

Test for POSITIVE relationship between:
--- increase in FA and GOAL-DIRECTEDNESS in: Putamen | group: SHORT ---

Cluster Index	Voxels	corrp1-p-MAX	corrp1-p-MAX X (vox)	corrp1-p-MAX Y (vox)	corrp1-p-MAX Z (vox)	corrp1-p-COG X (vox)	corrp1-p-COG Y (vox)	corrp1-p-COG Z (vox)	COPE-MAX	COPE-MAX X (vox)	COPE-

# *-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

# Statistics - Exploratory SUBGROUPS analysis (subgroups of habitual and goal-directed participants are inferred based on the behavioral measure)

### Prepare and arrange the behavioral data and the corresponding DTI folders data variables

In [19]:
# Get the behavioral data:
behav_clustered_all = pd.read_csv(os.path.join(behav_data_path, 'clustered_subgroups_HIS_May_2022.csv'))
# remove subjects that don't have DTI data
absent_from_DTI_data2 = [sub for sub in behav_clustered_all.ID if 'sub-' + str(sub) not in subjFoldersDTI]
behav_clustered_all = behav_clustered_all[~behav_clustered_all.ID.isin(absent_from_DTI_data2)]

# remove subjects that were excluded from the behavioral sub-groups clusters:
absent_from_behav_clusters = [sub for sub in subjFoldersDTI if int(sub.split('-')[1]) not in list(behav_clustered_all.ID)]
subjFoldersDTI_forSubgroups = [sub for sub in subjFoldersDTI if sub not in absent_from_behav_clusters]

# now make sure that subject lists are matching:
if not ['sub-' + str(sub) for sub in behav_clustered_all.ID] == subjFoldersDTI_forSubgroups:
    raise Exception('Subject lists are not matching')
    #pass # for debugging

# create variables foreach group seperately and for the subgroups seperately:
behav_clustered_short = behav_clustered_all[behav_clustered_all.ID < 200]
behav_clustered_long = behav_clustered_all[behav_clustered_all.ID > 200]

# sort the data according to the subgroups and create matching lists for the DTI data:
behav_clustered_short = behav_clustered_short.sort_values(by=['Cluster', 'ID']).reset_index(drop=True)
subjFoldersDTI_clustered_short = ['sub-' + str(sub) for sub in behav_clustered_short.ID]

behav_clustered_long = behav_clustered_long.sort_values(by=['Cluster', 'ID']).reset_index(drop=True)
subjFoldersDTI_clustered_long = ['sub-' + str(sub) for sub in behav_clustered_long.ID]


In [ ]:
subjFoldersDTI_clustered_short #goal-directed vs habitual - from behav_clustered_short
subjFoldersDTI_clustered_long #goal-directed vs habitual - from behav_clustered_long


### prepare the design matrix

In [ ]:
print('> Create the between group design matrix (of the calculated differences) - same is unpaired t-test (on the after-before differences)')
# create the folder
try:
    os.makedirs(os.path.join(models_path, 'sub_groups'), exist_ok=False)
    print('>> Created folder: ' + os.path.join(models_path, 'sub_groups'))
except:
    pass

# create the design matrix and contrasts:
N_goal_directed_in_clustered_1_DAY = behav_clustered_short.Cluster.value_counts()['Goal-directed']
N_habitual_in_clustered_1_DAY = behav_clustered_short.Cluster.value_counts()['Habitual']
print(f"design_ttest2 {os.path.join(models_path, 'sub_groups','design_unpaired_ttest_subgroups_in_1_Day')} {N_goal_directed_in_clustered_1_DAY} {N_habitual_in_clustered_1_DAY}")
os.system(f"design_ttest2 {os.path.join(models_path, 'sub_groups','design_unpaired_ttest_subgroups_in_1_Day')} {N_goal_directed_in_clustered_1_DAY} {N_habitual_in_clustered_1_DAY}")

N_goal_directed_in_clustered_3_DAY = behav_clustered_long.Cluster.value_counts()['Goal-directed']
N_habitual_in_clustered_3_DAY = behav_clustered_long.Cluster.value_counts()['Habitual']
print(f"design_ttest2 {os.path.join(models_path, 'sub_groups','design_unpaired_ttest_subgroups_in_3_Day')} {N_goal_directed_in_clustered_3_DAY} {N_habitual_in_clustered_3_DAY}")
os.system(f"design_ttest2 {os.path.join(models_path, 'sub_groups','design_unpaired_ttest_subgroups_in_3_Day')} {N_goal_directed_in_clustered_3_DAY} {N_habitual_in_clustered_3_DAY}")


### Create one file with each group's participants together

In [ ]:
for map_type in ['MD','FA']:          
    print(f"fslmerge -t {os.path.join(stats_path, map_type, f'SUBGROUPS_in_1_Day_AFTER_minus_BEFORE_{map_type}')} {' '.join([os.path.join(stats_path, map_type, f'{sub}_in-MNI_{map_type}_AFTER-minus-BEFORE.nii.gz') for sub in subjFoldersDTI_clustered_short])}")
    os.system(f"fslmerge -t {os.path.join(stats_path, map_type, f'SUBGROUPS_in_1_Day_AFTER_minus_BEFORE_{map_type}')} {' '.join([os.path.join(stats_path, map_type, f'{sub}_in-MNI_{map_type}_AFTER-minus-BEFORE.nii.gz') for sub in subjFoldersDTI_clustered_short])}")

    print(f"fslmerge -t {os.path.join(stats_path, map_type, f'SUBGROUPS_in_3_Day_AFTER_minus_BEFORE_{map_type}')} {' '.join([os.path.join(stats_path, map_type, f'{sub}_in-MNI_{map_type}_AFTER-minus-BEFORE.nii.gz') for sub in subjFoldersDTI_clustered_long])}")
    os.system(f"fslmerge -t {os.path.join(stats_path, map_type, f'SUBGROUPS_in_3_Day_AFTER_minus_BEFORE_{map_type}')} {' '.join([os.path.join(stats_path, map_type, f'{sub}_in-MNI_{map_type}_AFTER-minus-BEFORE.nii.gz') for sub in subjFoldersDTI_clustered_long])}")
    

### Settings

In [20]:
# a function to run randomise for the individual differences analyses:
def runRandomiseSubgroups(subgroups_in, map_type, region):
    print(f'Running randomise for sub-groups analysis:\n--- Subgroups in {subgroups_in} | {map_type} | {region} ---\n')
    #print(f'MAKE SURE TO REPLACE IN THE FUNCTION THE the print to os.system before running the command !!!\n')
    print(f"randomise_parallel \
        -i {os.path.join(stats_path, map_type, f'SUBGROUPS_in_{subgroups_in}_AFTER_minus_BEFORE_{map_type}')} \
        -o {os.path.join(stats_path, map_type, f'SUBGROUPS_in_{subgroups_in}_AFTER_minus_BEFORE_{map_type}_{region}')} \
        {'' if region=='whole_brain' else f'-m {masks_paths[region]}'} \
        -d {os.path.join(stats_path, 'models', 'sub_groups', f'design_unpaired_ttest_subgroups_in_{subgroups_in}.mat')} \
        -t {os.path.join(stats_path, 'models', 'sub_groups', f'design_unpaired_ttest_subgroups_in_{subgroups_in}.con')}  \
        -n 6000 -C 3.1")

# Get cluster info:
def getClusterInfoSubgroups(subgroups_in, map_type, region, ReversedSigThresh='0.90'):
    print(f'Testing stats for sub-groups analysis:\n--- Subgroups in {subgroups_in} | {map_type} | {region} ---\n')
    print(f'Goal-directed > Test Habitual \n' if subgroups_in in ['1_Day','3_Day'] else f'1-Day > 3-Day\n')
    print(os.popen(f'cluster \
        -i {os.path.join(stats_path, map_type, f"SUBGROUPS_in_{subgroups_in}_AFTER_minus_BEFORE_{map_type}_{region}_clusterm_corrp_tstat1.nii.gz")} \
        -c {os.path.join(stats_path, map_type, f"SUBGROUPS_in_{subgroups_in}_AFTER_minus_BEFORE_{map_type}_{region}_tstat1.nii.gz")}\
        -t {ReversedSigThresh} \
        --scalarname=corrp' + '"1-p"').read())

    print(f'Test Habitual > Goal-Directed \n' if subgroups_in in ['1_Day','3_Day'] else f'3-Day > 1-Day\n')
    print(os.popen(f'cluster \
        -i {os.path.join(stats_path, map_type, f"SUBGROUPS_in_{subgroups_in}_AFTER_minus_BEFORE_{map_type}_{region}_clusterm_corrp_tstat2.nii.gz")} \
        -c {os.path.join(stats_path, map_type, f"SUBGROUPS_in_{subgroups_in}_AFTER_minus_BEFORE_{map_type}_{region}_tstat2.nii.gz")}\
        -t {ReversedSigThresh} \
        --scalarname=corrp' + '"1-p"').read())


## IN SHORT TRAINING

In [22]:
subgroups_in = '1_Day'

### Whole brain - FA

In [23]:
map_type='FA'
region='whole_brain'

In [38]:
runRandomiseSubgroups(subgroups_in, map_type, region)

Running randomise for sub-groups analysis:
--- Subgroups in 1_Day | FA | whole_brain ---



### Putamen - FA

In [15]:
region='Putamen'

In [16]:
runRandomiseSubgroups(subgroups_in, map_type, region)

Running randomise for sub-groups analysis:
--- Subgroups in 1_Day | FA | Putamen ---



### Caudate - FA

In [18]:
region='Caudate'

In [19]:
runRandomiseSubgroups(subgroups_in, map_type, region)

Running randomise for sub-groups analysis:
--- Subgroups in 1_Day | FA | Caudate ---



### vmPFC - FA

In [21]:
region='vmPFC'

In [22]:
runRandomiseSubgroups(subgroups_in, map_type, region)

Running randomise for sub-groups analysis:
--- Subgroups in 1_Day | FA | vmPFC ---



### Whole brain - MD

In [24]:
map_type='MD'
region='whole_brain'

In [25]:
runRandomiseSubgroups(subgroups_in, map_type, region)

Running randomise for sub-groups analysis:
--- Subgroups in 1_Day | MD | whole_brain ---



### Putamen - MD

In [26]:
region='Putamen'

In [28]:
runRandomiseSubgroups(subgroups_in, map_type, region)

Running randomise for sub-groups analysis:
--- Subgroups in 1_Day | MD | Putamen ---



### Caudate - MD

In [28]:
region='Caudate'

In [31]:
runRandomiseSubgroups(subgroups_in, map_type, region)

Running randomise for sub-groups analysis:
--- Subgroups in 1_Day | MD | Caudate ---



### vmPFC - MD

In [25]:
region='vmPFC'

In [34]:
runRandomiseSubgroups(subgroups_in, map_type, region)

Running randomise for sub-groups analysis:
--- Subgroups in 1_Day | MD | vmPFC ---



## IN LONG TRAINING

In [23]:
subgroups_in = '3_Day'

### Whole brain - FA

In [24]:
map_type='FA'
region='whole_brain'

In [30]:
runRandomiseSubgroups(subgroups_in, map_type, region)

Running randomise for sub-groups analysis:
--- Subgroups in 3_Day | FA | whole_brain ---

randomise_parallel         -i /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/FA/SUBGROUPS_in_3_Day_AFTER_minus_BEFORE_FA         -o /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/FA/SUBGROUPS_in_3_Day_AFTER_minus_BEFORE_FA_whole_brain                  -d /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/models/sub_groups/design_unpaired_ttest_subgroups_in_3_Day.mat         -t /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/models/sub_groups/design_unpaired_ttest_subgroups_in_3_Day.con          -n 6000 -C 3.1


### Putamen - FA

In [16]:
region='Putamen'

In [17]:
runRandomiseSubgroups(subgroups_in, map_type, region)

Running randomise for sub-groups analysis:
--- Subgroups in 3_Day | FA | Putamen ---

randomise_parallel         -i /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/FA/SUBGROUPS_in_3_Day_AFTER_minus_BEFORE_FA         -o /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/FA/SUBGROUPS_in_3_Day_AFTER_minus_BEFORE_FA_Putamen         -m /export2/DATA/HIS/HIS_server/DTI_assitance_files/masks/Harvard-Oxford/Putamen-mask.nii.gz         -d /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/models/sub_groups/design_unpaired_ttest_subgroups_in_3_Day.mat         -t /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/models/sub_groups/design_unpaired_ttest_subgroups_in_3_Day.con          -n 6000 -C 3.1


### Caudate - FA

In [22]:
region='Caudate'

In [23]:
runRandomiseSubgroups(subgroups_in, map_type, region)

Running randomise for sub-groups analysis:
--- Subgroups in 3_Day | FA | Caudate ---

randomise_parallel         -i /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/FA/SUBGROUPS_in_3_Day_AFTER_minus_BEFORE_FA         -o /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/FA/SUBGROUPS_in_3_Day_AFTER_minus_BEFORE_FA_Caudate         -m /export2/DATA/HIS/HIS_server/DTI_assitance_files/masks/Harvard-Oxford/CaudateHead_Y-larger-than-1-mask.nii.gz         -d /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/models/sub_groups/design_unpaired_ttest_subgroups_in_3_Day.mat         -t /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/models/sub_groups/design_unpaired_ttest_subgroups_in_3_Day.con          -n 6000 -C 3.1


### vmPFC - FA

In [25]:
region='vmPFC'

In [26]:
runRandomiseSubgroups(subgroups_in, map_type, region)

Running randomise for sub-groups analysis:
--- Subgroups in 3_Day | FA | vmPFC ---

randomise_parallel         -i /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/FA/SUBGROUPS_in_3_Day_AFTER_minus_BEFORE_FA         -o /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/FA/SUBGROUPS_in_3_Day_AFTER_minus_BEFORE_FA_vmPFC         -m /export2/DATA/HIS/HIS_server/DTI_assitance_files/masks/Harvard-Oxford/vmPFC-mask.nii.gz         -d /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/models/sub_groups/design_unpaired_ttest_subgroups_in_3_Day.mat         -t /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/models/sub_groups/design_unpaired_ttest_subgroups_in_3_Day.con          -n 6000 -C 3.1


### Whole brain - MD

In [33]:
map_type='MD'
region='whole_brain'

In [34]:
runRandomiseSubgroups(subgroups_in, map_type, region)

Running randomise for sub-groups analysis:
--- Subgroups in 3_Day | MD | whole_brain ---

randomise_parallel         -i /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/MD/SUBGROUPS_in_3_Day_AFTER_minus_BEFORE_MD         -o /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/MD/SUBGROUPS_in_3_Day_AFTER_minus_BEFORE_MD_whole_brain                  -d /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/models/sub_groups/design_unpaired_ttest_subgroups_in_3_Day.mat         -t /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/models/sub_groups/design_unpaired_ttest_subgroups_in_3_Day.con          -n 6000 -C 3.1


### Putamen - MD

In [45]:
region='Putamen'

In [46]:
runRandomiseSubgroups(subgroups_in, map_type, region)

Running randomise for sub-groups analysis:
--- Subgroups in 3_Day | MD | Putamen ---

randomise_parallel         -i /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/MD/SUBGROUPS_in_3_Day_AFTER_minus_BEFORE_MD         -o /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/MD/SUBGROUPS_in_3_Day_AFTER_minus_BEFORE_MD_Putamen         -m /export2/DATA/HIS/HIS_server/DTI_assitance_files/masks/Harvard-Oxford/Putamen-mask.nii.gz         -d /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/models/sub_groups/design_unpaired_ttest_subgroups_in_3_Day.mat         -t /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/models/sub_groups/design_unpaired_ttest_subgroups_in_3_Day.con          -n 6000 -C 3.1


### Caudate - MD

In [54]:
region='Caudate'

In [49]:
runRandomiseSubgroups(subgroups_in, map_type, region)

Running randomise for sub-groups analysis:
--- Subgroups in 3_Day | MD | Caudate ---

randomise_parallel         -i /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/MD/SUBGROUPS_in_3_Day_AFTER_minus_BEFORE_MD         -o /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/MD/SUBGROUPS_in_3_Day_AFTER_minus_BEFORE_MD_Caudate         -m /export2/DATA/HIS/HIS_server/DTI_assitance_files/masks/Harvard-Oxford/CaudateHead_Y-larger-than-1-mask.nii.gz         -d /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/models/sub_groups/design_unpaired_ttest_subgroups_in_3_Day.mat         -t /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/models/sub_groups/design_unpaired_ttest_subgroups_in_3_Day.con          -n 6000 -C 3.1


### vmPFC - MD

In [56]:
region='vmPFC'

In [52]:
runRandomiseSubgroups(subgroups_in, map_type, region)

Running randomise for sub-groups analysis:
--- Subgroups in 3_Day | MD | vmPFC ---

randomise_parallel         -i /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/MD/SUBGROUPS_in_3_Day_AFTER_minus_BEFORE_MD         -o /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/MD/SUBGROUPS_in_3_Day_AFTER_minus_BEFORE_MD_vmPFC         -m /export2/DATA/HIS/HIS_server/DTI_assitance_files/masks/Harvard-Oxford/vmPFC-mask.nii.gz         -d /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/models/sub_groups/design_unpaired_ttest_subgroups_in_3_Day.mat         -t /export2/DATA/HIS/HIS_server/analysis/dwi_data/stats_alt_reg/models/sub_groups/design_unpaired_ttest_subgroups_in_3_Day.con          -n 6000 -C 3.1


## SUMMARY:

In [58]:
# Get cluster info:
for subgroups_in in ['1_Day','3_Day']:
    for map_type in ['FA','MD']:
        for region in ['whole_brain','Putamen','Caudate','vmPFC']:
            getClusterInfoSubgroups(subgroups_in, map_type, region, ReversedSigThresh='0.90')


Testing stats for sub-groups analysis:
--- Subgroups in 1_Day | FA | whole_brain ---

Goal-directed > Test Habitual 

Cluster Index	Voxels	corrp1-p-MAX	corrp1-p-MAX X (vox)	corrp1-p-MAX Y (vox)	corrp1-p-MAX Z (vox)	corrp1-p-COG X (vox)	corrp1-p-COG Y (vox)	corrp1-p-COG Z (vox)	COPE-MAX	COPE-MAX X (vox)	COPE-MAX Y (vox)	COPE-MAX Z (vox)	COPE-MEAN

Test Habitual > Goal-Directed 

Cluster Index	Voxels	corrp1-p-MAX	corrp1-p-MAX X (vox)	corrp1-p-MAX Y (vox)	corrp1-p-MAX Z (vox)	corrp1-p-COG X (vox)	corrp1-p-COG Y (vox)	corrp1-p-COG Z (vox)	COPE-MAX	COPE-MAX X (vox)	COPE-MAX Y (vox)	COPE-MAX Z (vox)	COPE-MEAN

Testing stats for sub-groups analysis:
--- Subgroups in 1_Day | FA | Putamen ---

Goal-directed > Test Habitual 

Cluster Index	Voxels	corrp1-p-MAX	corrp1-p-MAX X (vox)	corrp1-p-MAX Y (vox)	corrp1-p-MAX Z (vox)	corrp1-p-COG X (vox)	corrp1-p-COG Y (vox)	corrp1-p-COG Z (vox)	COPE-MAX	COPE-MAX X (vox)	COPE-MAX Y (vox)	COPE-MAX Z (vox)	COPE-MEAN

Test Habitual > Goal-Directed 

Cluster Ind